In [8]:
from pyspark import SparkConf, SparkContext
import pyspark
sc = SparkContext.getOrCreate(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [ ]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0][0], item[1][1]
  return (next_v, (prev_d + 1, prev_v))

def complete(item):
  v, old, new = item[0], item[1][0], item[1][1]
  return (v, (old[0] if old is not None else new[0], old[1] if old is not None else new[1]))

n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()


x = 12
d = 0
p = None

dest = 34

distances = sc.parallelize([(x, (d, p))]).partitionBy(n)

while True:       
    candidates = distances.filter(lambda i: i[1][0] == d).join(forward_edges, n).map(step)    

    new_distances = distances.fullOuterJoin(candidates, n).map(complete, True).persist(pyspark.StorageLevel.DISK_ONLY) 
    
    count = new_distances.filter(lambda i: i[1][0] == d + 1).count()
    path_v = new_distances.lookup(dest)
  
    if count > 0 and not path_v:
        d += 1
        distances = new_distances
        print("d = ", d, "count = ", count, "dest = ", path_v)
    else:               
        break

    


In [65]:
path=[]

if path_v:
    path.append(dest)
    while path_v and path_v[0][1]:
        path.append(path_v[0][1])
        path_v = new_distances.lookup(path_v[0][1])
   
print(",".join(str(x) for x in path[::-1]))
